# Pydantic and Annotated Types

Python provides a mechanism to attach metadata to any type.

In [1]:
from typing import Annotated

We simply define an annotated type this way:

In [2]:
SpecialInt = Annotated[int, "metadata 1", [1, 2, 3], 100]

Now, there is nothing special about this annotated type in Python natively. This is not a new type, it's still an `int`, but it happens to have some extra information attached to it.

Using Python, we can recover this metadata if we wanted to use it for our own purposes.

In [3]:
from typing import get_args

In [4]:
get_args(SpecialInt)

(int, 'metadata 1', [1, 2, 3], 100)

Pydantic can make use of these type annotations.

They need to be annotations that Pydantic can understand (i.e. we have to use specific objects in our annotations). This provides a very flexible way to add functionality to our types, such as constraints, validators, and much more.

Let's start with a simple example.

In [5]:
from pydantic import BaseModel, Field

In [6]:
class Model(BaseModel):
    x: int = Field(gt=0, le=100)
    y: int = Field(gt=0, le=100)
    z: int = Field(gt=0, le=100)

Note how the fields `x`, `y`, and `z` are basically the same type.

In [7]:
Model.model_fields

{'x': FieldInfo(annotation=int, required=True, metadata=[Gt(gt=0), Le(le=100)]),
 'y': FieldInfo(annotation=int, required=True, metadata=[Gt(gt=0), Le(le=100)]),
 'z': FieldInfo(annotation=int, required=True, metadata=[Gt(gt=0), Le(le=100)])}

Let's use annotations to define an annotated int, that includes the `Field` definition in the annotations.

In [8]:
BoundedInt = Annotated[int, Field(gt=0, le=100)]

As far as Python is concerned, this is just an `int` with some extra data attached to it. But Pydantic knows how to handled that `Field` object in the annotations.

In [9]:
class Model(BaseModel):
    x: BoundedInt
    y: BoundedInt
    z: BoundedInt

In [10]:
Model.model_fields

{'x': FieldInfo(annotation=int, required=True, metadata=[Gt(gt=0), Le(le=100)]),
 'y': FieldInfo(annotation=int, required=True, metadata=[Gt(gt=0), Le(le=100)]),
 'z': FieldInfo(annotation=int, required=True, metadata=[Gt(gt=0), Le(le=100)])}

As you can see the field definitions are identical, using annotations, to what we had before we used annotations.

And validation will occur as expected:

In [11]:
Model(x=10, y=20, z=30)

Model(x=10, y=20, z=30)

In [12]:
try:
    Model(x=0, y=10, z=103)
except ValueError as ex:
    print(ex)

2 validation errors for Model
x
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/greater_than
z
  Input should be less than or equal to 100 [type=less_than_equal, input_value=103, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/less_than_equal


As you can see, using annotated types can save us a lof of typing and help keep our code DRY (don't repeat yourself).

Technically, you use annotated types directly in a model - you don't have to define it first and then use inside a model.

In [13]:
class Model(BaseModel):
    field_1: Annotated[int, Field(gt=0)] = 1
    field_2: Annotated[str, Field(min_length=1, max_length=10)] | None = None

In [14]:
Model()

Model(field_1=1, field_2=None)

In [15]:
Model(field_1=10)

Model(field_1=10, field_2=None)

In [16]:
Model(field_2="Python")

Model(field_1=1, field_2='Python')

And of course validation works as expected:

In [17]:
try:
    Model(field_1=-10, field_2 = "Python" * 3)
except ValueError as ex:
    print(ex)

2 validation errors for Model
field_1
  Input should be greater than 0 [type=greater_than, input_value=-10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/greater_than
field_2
  String should have at most 10 characters [type=string_too_long, input_value='PythonPythonPython', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_too_long


We'll come back to using annotations in the context of validators and serializers soon.